In [1]:
import pickle
import emoji
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json


from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding


from keras import backend as K
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
K.set_session(session)


Using TensorFlow backend.


In [2]:
%ls weights/huffpo-v5

baseline-weights.hdf5  best-weights.hdf5


In [3]:
with open('hp-v5-tok.pickle', 'rb') as handle:
    hp_tok = pickle.load(handle)

In [4]:
with open('models/hp-model-v5.json','r') as handle:
    model_json = handle.read()
    huffpo_model = model_from_json(model_json)
huffpo_model.load_weights('weights/huffpo-v5/best-weights.hdf5')
print(huffpo_model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 18, 300)           3000600   
_________________________________________________________________
lstm_1 (LSTM)                (None, 500)               1602000   
_________________________________________________________________
dense_1 (Dense)              (None, 10002)             5011002   
Total params: 9,613,602
Trainable params: 9,613,602
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
%ls weights

bb-v1/    bb-v4/    huffpo-v1/    huffpo-v5.2/     huffpo-v6.1/
bb-v2/    bb-v5/    huffpo-v4/    huffpo-v5.3/     huffpo-v6.1-R2/
bb-v3/    bb-v5.1/  huffpo-v5/    huffpo-v5.3-R2/  huffpo-v6-R2/
bb-v3.1/  bb-v5.2/  huffpo-v5.1/  huffpo-v6/


In [6]:
with open('hp-v5.2-tok.pickle', 'rb') as handle:
    hp_bilstm_tok = pickle.load(handle)
    
with open('models/hp-model-v5.2.json','r') as handle:
    model_json = handle.read()
    huffpo_bilstm = model_from_json(model_json)
huffpo_bilstm.load_weights('weights/huffpo-v5.2/best-weights.hdf5')
print(huffpo_bilstm.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 18, 300)           3000600   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 500)               3204000   
_________________________________________________________________
dense_1 (Dense)              (None, 10002)             5011002   
Total params: 11,215,602
Trainable params: 11,215,602
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
with open('hp-v5.2-tok.pickle', 'rb') as handle:
    hp_bilstm_tok = pickle.load(handle)
    
with open('models/hp-model-v5.3.json','r') as handle:
    model_json = handle.read()
    huffpo_bilstm_v53 = model_from_json(model_json)
huffpo_bilstm_v53.load_weights('weights/huffpo-v5.3-R2/best-weights.hdf5')
print(huffpo_bilstm_v53.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 6, 300)            3000600   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1000)              3204000   
_________________________________________________________________
dense_1 (Dense)              (None, 10002)             10012002  
Total params: 16,216,602
Trainable params: 16,216,602
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
# generate a sequence from a language model
#seed_text is str
MAX_LEN = 6


def generate_seq(model, tokenizer, max_length, seed_text, n_words):
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])
        # pre-pad sequences to a fixed length
        encoded = pad_sequences(encoded, maxlen=max_length, padding='pre')
        pred_vector = model.predict(encoded, verbose=0)[0] #1 x 10002 vector of probabiltiies 
        class_preds = np.argsort(pred_vector)
        class_preds = np.flip(class_preds) #1 x 10002 vector of word indexes, ordered from most-likely
        top_pred = class_preds[0]
        
        if top_pred in tokenizer.index_word:
            
            nxt_word = tokenizer.index_word[top_pred]

        else:
            nxt_word = ' ??? '
        """
        fix me. Almost there!
        
        """
        
 


        in_text += ' ' + nxt_word
    return in_text


In [14]:
generate_seq(huffpo_model, hp_tok, 18, "trump", 15)

'trump 2020 oov same same thing bro ok oov oov oov lol ❤️ again oov oov'

In [16]:
generate_seq(huffpo_bilstm_v53, hp_bilstm_tok, 6, "trump", 15)

'trump 2016 oov oov oov oov oov oov oov oov oov oov oov oov oov oov'

In [ ]:
"""
INPUT_STR = 'trump is'
test = hp_bilstm_tok.texts_to_sequences([INPUT_STR])
encoded_input = pad_sequences(test, maxlen=MAX_LEN, padding='pre')
pred_vector = huffpo_bilstm.predict(encoded_input, verbose=0)[0] #1 x 10002 vector of probabiltiies 
class_preds = np.argsort(pred_vector)
class_preds = np.flip(class_preds) #1 x 10002 vector of word indexes, ordered from most-likely
top_pred = class_preds[0]
nxt_word = hp_bilstm_tok.index_word[top_pred]
#print(len(pred_code))

#pred_word = hp_bilstm_tok.sequences_to_texts(pred_code)
#print(pred_word)

"""